# ACTUS CAPFL Contract: Interest Rate Cap

This notebook demonstrates the **Cap/Floor (CAPFL)** contract type from the ACTUS standard.

## What is a CAPFL Contract?

A Cap/Floor contract protects against interest rate movements:
- **Interest Rate Cap**: Protects borrowers from rates going too high
- **Interest Rate Floor**: Protects lenders from rates going too low

## Example: Interest Rate Cap on Floating Rate Loan

Scenario:
- Company has $10M floating rate loan at LIBOR + 2%
- Wants protection if LIBOR exceeds 5%
- Buys interest rate cap with 5% strike
- Term: 5 years, quarterly resets

In [ ]:
from jactus.contracts import create_contract
from jactus.core import (
    ContractAttributes,
    ContractType,
    ContractRole,
    ActusDateTime,
    ContractState,
    ContractPerformance,
)
from jactus.observers import ConstantRiskFactorObserver, MockChildContractObserver
import jax.numpy as jnp

## Step 1: Create Underlier (Floating Rate Swap)

In [ ]:
# Contract dates
status_date = ActusDateTime(2024, 1, 1, 0, 0, 0)
maturity_date = ActusDateTime(2029, 1, 1, 0, 0, 0)  # 5 years

# Create swap state (floating rate leg)
swap_state = ContractState(
    tmd=maturity_date,
    sd=status_date,
    nt=jnp.array(10_000_000.0, dtype=jnp.float32),  # $10M notional
    ipnr=jnp.array(0.03, dtype=jnp.float32),  # Current floating rate 3%
    ipac=jnp.array(0.0, dtype=jnp.float32),
    feac=jnp.array(0.0, dtype=jnp.float32),
    nsc=jnp.array(1.0, dtype=jnp.float32),
    isc=jnp.array(1.0, dtype=jnp.float32),
    prf=ContractPerformance.PF,
)

# Register swap with child observer
child_observer = MockChildContractObserver()
child_observer.register_child("SWAP-FLOAT", state=swap_state, events=[])

print(f"Floating Rate Loan:")
print(f"  Notional: ${float(swap_state.nt):,.0f}")
print(f"  Current Rate: {float(swap_state.ipnr)*100:.1f}%")

## Step 2: Create Interest Rate Cap

In [ ]:
# Create cap attributes
cap_attrs = ContractAttributes(
    contract_id="CAP-5PCT",
    contract_type=ContractType.CAPFL,
    contract_role=ContractRole.RPA,  # Long cap (protection buyer)
    status_date=status_date,
    maturity_date=maturity_date,
    notional_principal=10_000_000.0,
    rate_reset_cap=0.05,  # 5% cap rate
    rate_reset_cycle="3M",  # Quarterly
    currency="USD",
    contract_structure='{"Underlier": "SWAP-FLOAT"}',
)

# Create risk factor observer
rf_observer = ConstantRiskFactorObserver(constant_value=0.03)

# Create cap contract
cap = create_contract(cap_attrs, rf_observer, child_observer)

print(f"\nInterest Rate Cap Created:")
print(f"  Notional: ${cap_attrs.notional_principal:,.0f}")
print(f"  Cap Rate: {cap_attrs.rate_reset_cap*100}%")
print(f"  Reset Frequency: Quarterly")
print(f"  Term: 5 years")

## Step 3: Understand Cap Payoff

At each reset date:
- If floating rate > cap rate: Payoff = (Floating Rate - Cap Rate) × Notional × Period
- If floating rate ≤ cap rate: Payoff = 0

The cap effectively limits the maximum interest rate paid.

In [ ]:
# Simulate cap
cap_result = cap.simulate(rf_observer)

print(f"\nCap Simulation Results:")
print(f"Total events: {len(cap_result.events)}")
print(f"Expected: ~{5*4} quarterly resets over 5 years")

## Step 4: Calculate Protection Value

Let's see what happens when rates exceed the cap

In [ ]:
# Example: Calculate payoff if rate rises to 7%
notional = 10_000_000
cap_rate = 0.05  # 5%
market_rate = 0.07  # 7%
period = 0.25  # Quarterly = 3 months / 12 months

# Without cap: Interest cost
interest_no_cap = notional * market_rate * period

# With cap: Effective interest cost
interest_with_cap = notional * cap_rate * period

# Cap payoff (compensation)
cap_payoff = notional * (market_rate - cap_rate) * period

print("\nScenario: Market Rate Rises to 7%")
print("=" * 50)
print(f"Without Cap:")
print(f"  Quarterly Interest: ${interest_no_cap:,.2f}")
print(f"  Annual Interest: ${interest_no_cap * 4:,.2f}")
print(f"\nWith 5% Cap:")
print(f"  Quarterly Interest: ${interest_with_cap:,.2f}")
print(f"  Cap Payoff: ${cap_payoff:,.2f}")
print(f"  Net Cost: ${interest_with_cap:,.2f}")
print(f"\nSavings: ${cap_payoff:,.2f} per quarter")
print(f"Annual Savings: ${cap_payoff * 4:,.2f}")

## Step 5: Visualize Cap Protection

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Range of possible interest rates
rates = np.linspace(0.01, 0.10, 100)  # 1% to 10%
cap_rate = 0.05
notional = 10_000_000
period = 0.25

# Calculate costs
interest_costs = notional * rates * period
cap_payoffs = np.maximum(rates - cap_rate, 0) * notional * period
net_costs = interest_costs - cap_payoffs

# Create plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Interest Cost
ax1.plot(rates*100, interest_costs/1000, linewidth=2, color='red', 
         label='Without Cap', linestyle='--')
ax1.plot(rates*100, net_costs/1000, linewidth=2, color='green', 
         label='With Cap (Net Cost)')
ax1.axvline(x=cap_rate*100, color='blue', linestyle=':', 
            label=f'Cap Rate = {cap_rate*100}%', linewidth=2)
ax1.set_xlabel('Market Interest Rate (%)', fontsize=11)
ax1.set_ylabel('Quarterly Interest Cost ($1000s)', fontsize=11)
ax1.set_title('Interest Rate Cap Protection', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Cap Payoff
ax2.fill_between(rates*100, 0, cap_payoffs/1000, alpha=0.3, color='green')
ax2.plot(rates*100, cap_payoffs/1000, linewidth=2, color='green')
ax2.axvline(x=cap_rate*100, color='blue', linestyle=':', 
            label=f'Cap Rate = {cap_rate*100}%', linewidth=2)
ax2.set_xlabel('Market Interest Rate (%)', fontsize=11)
ax2.set_ylabel('Cap Payoff ($1000s)', fontsize=11)
ax2.set_title('Interest Rate Cap Payoff', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Charts show:")
print(f"  • Left: Cap limits maximum interest cost to {cap_rate*100}%")
print(f"  • Right: Cap pays off when rate exceeds {cap_rate*100}%")

## Key Observations

The CAPFL contract type implements:

1. **Interest Rate Protection**:
   - Cap: Limits maximum rate paid
   - Floor: Guarantees minimum rate received (not shown)

2. **Payoff Structure**:
   - Cap Payoff = max(Rate - Cap Strike, 0) × Notional × Period
   - Floor Payoff = max(Floor Strike - Rate, 0) × Notional × Period

3. **Use Cases**:
   - Hedging floating rate loans
   - Protecting investment returns
   - Managing interest rate risk

4. **ACTUS Compliance**: Follows ACTUS v1.1 specification

## References

- ACTUS Specification v1.1, Section 7.14 - Cap/Floor (CAPFL)
- [ACTUS Technical Specification](https://www.actusfrf.org/)